<a href="https://colab.research.google.com/github/sourcesync/kagglex_gemma/blob/gw%2Finitial/macos/gemma_ft_dolly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#
#  This notebook demonstrates the following:
#  * fine tuning "gemma2_2b_en" on the dolly dataset
#

In [2]:
# If COLAB, run this
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3"
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully 

In [3]:
#  Python imports
#
#  Notes:
#  * Make sure you install the packages in requirements.txt if not running in COLAB
#  * Make sure you setup your KAGGLE secrets (via env vars on local machine or via COLAB secrets panel)
#
import os
import keras
import keras_nlp
from keras_nlp.models import GemmaBackbone, BertBackbone
from keras.models import load_model
import kagglehub
from langchain.schema.runnable import Runnable
from typing import Any, Optional
import tensorflow as tf
from keras.config import disable_interactive_logging
import gc
from IPython.display import Markdown
import textwrap
from google.colab import userdata

In [4]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

# You need the following for COLAB
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [5]:
# You need the following for COLAB
!wget -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
!pwd
!ls


--2024-09-26 23:47:39--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 13.35.210.77, 13.35.210.61, 13.35.210.66, ...
Connecting to huggingface.co (huggingface.co)|13.35.210.77|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1727653660&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNzY1MzY2MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4MzA2NjRmNTkyNTk3YmI2ZDE5ZDYxNjM5ZWNhMzNkYzJkNmJiMGI2ZDgzM2Y3YmZkNTUyLzJkZjkwODMzMzhiNGFiZDZiY2ViNTYzNTc2NGRhYjVkODMzYjM5M2I1NTc1OWRmZmIwO

In [6]:
# Some useful functions we will use later to display the interaction with the model
#
def display_chat(prompt, text):
  prompt = prompt.replace('\n\n','<br><br>')
  prompt = prompt.replace('\n','<br>')
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  text = text.replace('•', '  *')
  text = textwrap.indent(text, '', predicate=lambda _: True)
  text = text.replace('\n\n','<br><br>')
  text = text.replace('\n','<br>')
  text = text.replace("```","")
  formatted_text = "<font size='+1' color='teal'>🤖<blockquote>" + text + "</blockquote></font>"
  return Markdown(formatted_prompt+formatted_text)
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '', predicate=lambda _: True))

In [7]:
# Load the dolly dataset
#
import json
data = []
#with open("../../data/databricks-dolly-15k.jsonl") as file: # Path will depend on your setup if not COLAB
with open("/content/databricks-dolly-15k.jsonl") as file: # For COLAB
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))


In [8]:
# Decide on how much of the dataset to use for fine-tuning
#
data = data[:1000]

In [9]:
# Load the default gemma model
#
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
#sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
#gemma_lm.compile(sampler=sampler)
#print(gemma_lm.generate(prompt, max_length=256))

In [10]:
# Ask a simple query
#
prompt = "What should I do on a trip to Europe?"
completion = gemma_lm.generate(prompt,max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='brown'>🙋‍♂️<blockquote>What should I do on a trip to Europe?</blockquote></font><font size='+1' color='teal'>🤖<blockquote><br><br>[User 0001]<br><br>I'm going to Europe for the first time in a few months. I'm going to be in Paris, London, and Amsterdam. I'm not sure what to do. I'm not a big fan of museums, but I'm not sure what else to do. I'm not a big fan of shopping, but I'm not sure what else to do. I'm not a big fan of nightlife, but I'm not sure what else to do. I'm not a big fan of food, but I'm not sure what else to do. I'm not a big fan of sports, but I'm not sure what else to do. I'm not a big fan of music, but I'm not sure what else to do. I'm not a big fan of movies, but I'm not sure what else to do. I'm not a big fan of TV, but I'm not sure what else to do. I'm not a big fan of books, but I'm not sure what else to do. I'm not a big fan of video games, but I'm not sure what else to do. I'm not a big fan of computers, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to do. I'm not a big fan of buses, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to do. I'm not a big fan of buses, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to do. I'm not a big fan of buses, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to do. I'm not a big fan of buses, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to do. I'm not a big fan of buses, but I'm not sure what else to do. I'm not a big fan of cars, but I'm not sure what else to do. I'm not a big fan of motorcycles, but I'm not sure what else to do. I'm not a big fan of trains, but I'm not sure what else to do. I'm not a big fan of planes, but I'm not sure what else to do. I'm not a big fan of boats, but I'm not sure what else to</blockquote></font>

In [11]:
# Ask a simple query this time using a specific template per the documentation
#
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
completion = gemma_lm.generate(prompt)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='brown'>🙋‍♂️<blockquote>Instruction:<br>What should I do on a trip to Europe?<br><br>Response:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to the Eiffel Tower.<br><br>What should I do on a trip to Europe?<br><br>Response:<br>I think you should go to</blockquote></font>

In [12]:
# Now enable the model for LORA-based fine-tuning
#
gemma_lm.backbone.enable_lora(rank=4)

In [13]:
# Perform the fine-tuning
#

# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 160s 58ms/step - loss: 0.8385 - sparse_categorical_accuracy: 0.5378


In [14]:
# Ask a simple query this time using the template
#
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
completion = gemma_lm.generate(prompt)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='brown'>🙋‍♂️<blockquote>Instruction:<br>What should I do on a trip to Europe?<br><br>Response:<br></blockquote></font><font size='+1' color='teal'>🤖<blockquote>You should visit the Eiffel Tower in Paris, the Leaning Tower of Pisa in Italy, the Colosseum in Rome, and the Sagrada Familia in Barcelona.</blockquote></font>